<a href="https://colab.research.google.com/github/itamarco/face-mask-detector/blob/master/face_masks_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

import keras
from keras import layers
from keras.models import Sequential, Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score
import pathlib

import imutils



Mount Google Drive

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd 'drive/MyDrive/face-mask-dataset/'
 
!ls

/content/drive/MyDrive/face-mask-dataset
dataset  face-model-t1.h5  with_mask.zip  without_mask.zip


Unzip train and test datasets

In [ ]:
!unzip -q ./with_mask.zip -d ./dataset
!unzip -q ./without_mask.zip -d ./dataset

!echo with_mask images: `ls ./dataset/with_mask | wc -l`
!echo without_mask images: `ls ./dataset/without_mask | wc -l`

## Parameters

In [15]:
BATCH_SIZE = 10
EPOCHS = 10
IMG_HEIGHT = 140
IMG_WIDTH = 140

## Image Data Generation/Augmentation:

In [16]:
DATA_DIR = './dataset'
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.3,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = datagen.flow_from_directory(
    DATA_DIR,
    batch_size=BATCH_SIZE, 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset='training'
)


val_ds = datagen.flow_from_directory(
    DATA_DIR,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset='validation'
)

Found 2684 images belonging to 2 classes.
Found 1149 images belonging to 2 classes.


`Dataset.cache()` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training the model. 

`Dataset.prefetch()` overlaps data preprocessing and model execution while training.

##Build the network

This is a convolution network:
* Two pairs of Conv and MaxPool layers to extract features from the dataset.
* Flatten and Dropout layer to convert the data in 1D and ensure overfitting.
* Two Dense layers for classification.

In [17]:
model = Sequential([
    layers.Conv2D(100, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(100, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dropout(0.5),
    
    layers.Dense(50, activation='relu'),
    layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 138, 138, 100)     2800      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 69, 69, 100)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 67, 67, 100)       90100     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 33, 33, 100)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 108900)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 108900)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)               

Initialize a callback checkpoint to keep saving best model after each epoch while training:

In [ ]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

## Train the Model

In [18]:
model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[]
)
model.save('face-mask-model.h5')
print('Training is Done!')

Epoch 1/10
200/269 [=====================>........] - ETA: 3:01 - loss: 0.6047 - acc: 0.6424

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


269/269 [==============================] - 1017s 4s/step - loss: 0.5652 - acc: 0.6774 - val_loss: 0.4409 - val_acc: 0.7972
Epoch 2/10
269/269 [==============================] - 283s 1s/step - loss: 0.3406 - acc: 0.8545 - val_loss: 0.2398 - val_acc: 0.9104
Epoch 3/10
269/269 [==============================] - 280s 1s/step - loss: 0.3261 - acc: 0.8636 - val_loss: 0.3383 - val_acc: 0.8729
Epoch 4/10
269/269 [==============================] - 280s 1s/step - loss: 0.3302 - acc: 0.8604 - val_loss: 0.2447 - val_acc: 0.9156
Epoch 5/10
269/269 [==============================] - 282s 1s/step - loss: 0.3030 - acc: 0.8742 - val_loss: 0.2288 - val_acc: 0.9060
Epoch 6/10
269/269 [==============================] - 281s 1s/step - loss: 0.2952 - acc: 0.8845 - val_loss: 0.2184 - val_acc: 0.9121
Epoch 7/10
269/269 [==============================] - 279s 1s/step - loss: 0.3448 - acc: 0.8609 - val_loss: 0.2080 - val_acc: 0.9260
Epoch 8/10
269/269 [==============================] - 281s 1s/step - loss: 0.26